## 3.4 電影評論分類：二分類問題
本節使用IMDB數據集，其包含 IMDB 中50000條嚴重兩極化的評論。數據集被分為用於訓練的25000條評論，測試集和訓練集都包含50%的正面評論和50%的負面評論。實現如下：


In [ ]:
# 3-1 加載IMDB數據集
'''
參數num_words = 20000的意思是，
僅保留訓練數據中前20000個最常見的單詞，
低發生率的單詞將被捨棄，
這樣得到的向量數據不會太大，便於處理。

而train_data和test_data這兩個變量都式評論組成的list；
每條評論又是單詞索引組成的list( 表示一系列單詞 )，
train_labels和test_labels都是0(負面)和1(正面)組的list。
'''

from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
    num_words=20000)
print(train_data[0])
print(train_labels[20000])

###
# 由於限定前10000個最常見的單詞，單詞索引都不會超過20000個
# 因為索引值通常為位移值
###

print(max([max(sequence)for sequence in train_data]))

###
# 下面這段代碼有個很酷炫的功能
# 可以將某條評論迅速解碼為英文單詞
###

word_index = imdb.get_word_index()
reverse_word_index = dict(
    [(value, key) for (key, value) in word_index.items()]
)
decoded_review = ' '.join(
    [reverse_word_index.get(i - 3, '?') for i in train_data[0]]
)
	
###
# word_index是一個將單詞映射為整數索引的字典
# 而dict的第一行之key、value值顛倒，將整數索引映射為單詞
# 注意，最後一段程式碼為：
# 將評論解碼。注意，索引減去了3，因為0、1、2
# 分別為'padding'(填充)、'start of sequence'(序列開始)、
# 'unknown'(未知詞)，所分別保留的索引值
###


### 3.4.1 準備數據
我們不能將整數序列直接輸入NN，需要將列表轉換為張量，轉換方式有兩種，如下：<br>
* 填充列表( list )：使其具有相同的長度，再將列表轉換成形狀為( samples, word_indices )的整數張量，然後網路第一層使用能處理這種整數張量的層( 即Embedding層，後續章節會做討論 )。
* 對列表進行one-hot編碼：將其轉換為只有0和1組成的向量。例如，序列[3,5]轉換為20000維的向量，只有索引為3、5的元素是1，其餘元素都是0。<br>
而下面我們採用第二種方法進行數據向量化，如下所示：


In [ ]:
# 3-2 將整數序列編碼為二進制矩陣
import numpy as np
import pprint

def vectorize_sequences(sequences, dimension=20000):
    results = np.zeros((len(sequences), dimension))          # 創建一個形狀為(len(sequences), dimension)的零矩陣
    for i, sequence in enumerate(sequences):                 # i 為索引值， sequence為索引值的list內容
        results[i, sequence] = 1.                            # result[i]的指定索引設為1
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

pprint.pprint(x_train[0])       # 訓練樣本現在變成了這樣
pprint.pprint(x_test)           # 測試樣本現在變成了這樣

###
# 現在，我們將標籤向量化
# 然後我們就可以將數據加入NN中了
###

y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')


### 3.4.2 構建網路
輸入數據是向量，標籤是張量(1和0)，這是我們遇到最簡單的情況。有一類網路在這種問題上表現很好，就是帶有relu激活的全連接層( Dense )的簡單堆疊。<br><br>

對於這種Dense層的堆疊，我們需要以下兩個關鍵架構：
* 網路有多少層
* 每層有多少個隱藏單元 <br><br>

注意，隱藏單元數越多，網路越能學到更加複雜的表示，但網路的計算代價也變得更大，且可能會導致學到不好的模式(這種模式會提高訓練數據上的性能，但不會提高測試數據上的性能，即overfitting)。<br><br>

下面，我們將以Keras實現如圖3-6所示，三層網路之程式碼。<br>



In [ ]:
# 3-3 模型定義
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(20000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


In [ ]:
# 3-4 編譯模型
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
# 3-5 配置優化器

###
# 3-4代碼為將優化器、損失函數、指標作為字符串輸入，
# 這是因為rmsprop, binary_crossentropy和accuracy
# 都是keras內置的一部分
# 若我們想要引入自定義的函數，
# 前者可以向optimizer參數傳入一個優化類實例來實現
# 後者可以通過向loss、metrics參數傳入函數對向來實現
###

from keras import optimizers

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
# 3-6 使用自定義的損失函數和衡量指標
from keras import losses
from keras import metrics

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss=losses.binary_crossentropy,
              metrics=[metrics.binary_accuracy])


### 3.4.4 驗證我們的方法
為了在訓練過程中監控模型在前所未見的數據上的準確度，我們需要將原始數據流出10000個樣本作為驗證集。


In [ ]:
# 3-7 切割出驗證集
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]


In [ ]:
# 3-8 訓練模型

###
# 現在使用512個樣本組成的小批量
# 將模型訓練20個輪次(epoch)(即對x_train和y_tran，
# 兩個張量的所有樣本進行20次迭代)。
# 與此同時，我們還要監控在切割出的10000個樣本上的損失值
# 與準確度，我們可以通過將驗證數據傳入validation_data參數來完成
###

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))


In [ ]:

###
# 注意，調用model.fit()返回了一個History對象
# 這個對象有一個成員history，他是一個字典，
# 包含了訓練過程中的所有數據，接下來，我們來瞧瞧他的樣子
###

history_dict = history.history
history_dict.keys()

###
# 字典包含了4個條目，分別對應驗證過程和訓練過程中所監控的指標
# 接下來，我們來看看兩者的損失值以及準確度的差異。
###


In [ ]:
# 3-9 繪製訓練損失和驗證損失

import matplotlib.pyplot as plt

history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, len(loss_values) + 1)

plt.plot(epochs, loss_values, 'bo', label='Training loss')          # 'bo'表示藍色圓點
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')     # 'b'表示藍色實線
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()


In [ ]:
# 3-10 繪製訓練經度和驗證經度

plt.clf()                           # 清空圖像
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

plt.plot(epochs, acc_values, 'bo', label='Training acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

###
# 我們從圖中的結果可以知，
# 模型在訓練數據上的表現越來越好，
# 但是在前所未見的數據上不一定表現得越來越好
# 也就是說，這是過擬和現象(overfitting)，
# 最終學到的表示僅針對於訓練數據
# 無法泛化到訓練集之外的數據
###


In [ ]:
# 3-11 從頭開始訓練一個模型

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(20000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=4, batch_size=512)
results = model.evaluate(x_test, y_test)

print(results)
print(model.predict(x_test))

###
# 這種相當簡單的方法得到了88%的準確度
# 而由model.predict(x_test)回傳的結果可知
# 網路對某些樣本的結果非常確信(大於等於0.999)
###


### 3.4.5 進一步的實驗
通過以下實驗，我們可以確認前面選擇的網路架構是非常合理的，雖然仍有改進的空間。<br>
* 前面使用了兩個隱藏層，你可以嘗試使用一個或三個隱藏層，然後觀察對驗證準確度和測試準確度的影響。
* 嘗試使用更多或更少的隱藏單元，比如32個、64個。
* 嘗試使用mse損失函數代替binary_crossentropy。
* 嘗試使用tanh，代替relu。


### 3.4.6 小結
下面是我們應該從這個例子中學到的重點。<br>
* 通常需要對原始數據進行大量預處理，以便將其轉換為張量輸入到神經網路中，單詞序列可以編碼為二進制向量，當然，也有其他編碼方式。
* 帶有relu繳活的Dense層堆疊，可以解決很多問題(包括情感分類)，我們可能會經常用到這種類型。
* 對於二分類問題(兩個輸出類別)，網路的最後一層應該是只有一個單元並使用sigmoid激活的Dense層，網路輸出應該是0-1範圍內的標量。
* 對於二分類問題的sigmoid標量輸出，我們應該使用binary_crossentropy損失函數。
* 隨著NN在訓練數據上的表現越來越好，模型最終會過擬合，並在前所未見的數據上得到越來越差的結果。因此，我們一定要一直監控模型在訓練集之外的數據上的性能。

